In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.applications.efficientnet import preprocess_input
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import log_loss, classification_report, confusion_matrix, roc_curve, auc
from tensorflow.keras.applications import EfficientNetB0
import seaborn as sns
from datetime import datetime

# Кастомный callback для записи learning rate
class LRTracker(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = tf.keras.backend.get_value(self.model.optimizer.learning_rate)

# Пути к данным
train_dir = "data/train/train"
val_dir = "data/test/test"

# Загрузка и проверка данных
def load_and_validate_data():
    train_images = [os.path.join(train_dir, img) for img in os.listdir(train_dir) if img.endswith(('.jpg', '.png'))]
    val_images = [os.path.join(val_dir, img) for img in os.listdir(val_dir) if img.endswith(('.jpg', '.png'))]

    assert len(train_images) > 0, "Нет изображений в train"
    assert len(val_images) > 0, "Нет изображений в test"

    return train_images, val_images

# Улучшенная функция загрузки изображений
def load_images(image_paths):
    images, labels = [], []
    for img_path in image_paths:
        try:
            img = cv2.imread(img_path)
            if img is None:
                continue
            img = cv2.resize(img, (224, 224))  # EfficientNet стандартный размер
            img = preprocess_input(img)  # Специфичная предобработка
            images.append(img)
            labels.append(1 if "dog" in os.path.basename(img_path).lower() else 0)
        except Exception as e:
            print(f"Ошибка загрузки {img_path}: {str(e)}")
    return np.array(images), np.array(labels)

# Загрузка данных
train_images, val_images = load_and_validate_data()
X_train, y_train = load_images(train_images)
X_val, y_val = load_images(val_images)

# Проверка баланса классов
print(f"\nРаспределение классов в train: {np.bincount(y_train)}")
print(f"Распределение классов в val: {np.bincount(y_val)}")

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

# Улучшенная аугментация с препроцессингом
train_datagen = ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    brightness_range=[0.85, 1.15],
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator()

# Веса классов для борьбы с дисбалансом
class_weights = {0: len(y_train)/(2*np.bincount(y_train)[0]),
                 1: len(y_train)/(2*np.bincount(y_train)[1])}

# Создание модели
def create_model():
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Замораживаем только первые 3 слоя
    for layer in base_model.layers[:3]:
        layer.trainable = False

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(1024, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(512, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])

    optimizer = AdamW(learning_rate=0.001, weight_decay=1e-4)
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy',
                 tf.keras.metrics.Precision(name='precision'),
                 tf.keras.metrics.Recall(name='recall'),
                 tf.keras.metrics.AUC(name='auc')]
    )
    return model

model = create_model()

# Callbacks
log_dir = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
callbacks = [
    EarlyStopping(monitor='val_auc', patience=5, mode='max', verbose=1, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_auc', save_best_only=True, mode='max'),
    ReduceLROnPlateau(monitor='val_auc', factor=0.5, patience=3, min_lr=1e-6, mode='max', verbose=1),
    LRTracker(),
    TensorBoard(log_dir=log_dir)
]

# Обучение модели (уменьшено до 30 эпох)
history = model.fit(
    train_datagen.flow(X_train, y_train, batch_size=32, shuffle=True),
    epochs=30,  # Уменьшено с 100 до 30
    validation_data=val_datagen.flow(X_val, y_val, batch_size=32),
    callbacks=callbacks,
    class_weight=class_weights,
    verbose=1
)

# Визуализация
def plot_metrics(history):
    plt.figure(figsize=(18, 12))

    metrics = ['loss', 'accuracy', 'precision', 'recall', 'auc']
    for i, metric in enumerate(metrics):
        plt.subplot(2, 3, i+1)
        plt.plot(history.history[metric], label=f'Train {metric}')
        plt.plot(history.history[f'val_{metric}'], label=f'Validation {metric}')
        plt.xlabel('Epochs')
        plt.ylabel(metric)
        plt.legend()

    plt.subplot(2, 3, 6)
    plt.plot(history.history['lr'], label='Learning Rate')
    plt.xlabel('Epochs')
    plt.ylabel('LR')
    plt.yscale('log')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_metrics(history)

# Оценка модели с добавлением LogLoss
def evaluate_model(model, X_test, y_test):
    test_generator = val_datagen.flow(X_test, y_test, batch_size=32, shuffle=False)

    # Предсказания с оптимальным порогом
    y_pred = model.predict(test_generator)
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]

    print(f"\nОптимальный порог: {optimal_threshold:.4f}")
    y_pred_classes = (y_pred > optimal_threshold).astype(int)

    # Вычисление LogLoss
    test_loss = log_loss(y_test, y_pred)
    print(f"\nLogLoss на тестовых данных: {test_loss:.4f}")

    # Метрики
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred_classes, target_names=['Cat', 'Dog']))

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred_classes)
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Cat', 'Dog'],
                yticklabels=['Cat', 'Dog'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

    # ROC Curve
    roc_auc = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()

evaluate_model(model, X_test, y_test)

# Функция предсказания с оптимальным порогом
def predict_image(img_path, model, threshold=None):
    try:
        img = cv2.imread(img_path)
        if img is None:
            raise ValueError(f"Изображение {img_path} не загружено")

        img = cv2.resize(img, (224, 224))
        img = preprocess_input(img)
        img_array = np.expand_dims(img, axis=0)

        # Автоматическое определение порога
        if threshold is None:
            test_generator = val_datagen.flow(X_test, y_test, batch_size=32, shuffle=False)
            y_pred = model.predict(test_generator)
            fpr, tpr, thresholds = roc_curve(y_test, y_pred)
            threshold = thresholds[np.argmax(tpr - fpr)]

        probability = model.predict(img_array, verbose=0)[0][0]
        class_name = 'Dog' if probability > threshold else 'Cat'

        return {
            'class': class_name,
            'probability': float(probability),
            'threshold': float(threshold),
            'confidence': float(probability if class_name == 'Dog' else 1 - probability)
        }
    except Exception as e:
        print(f"Ошибка при предсказании: {str(e)}")
        return None

# Пример использования
sample_image = val_images[0] if val_images else train_images[0]
prediction = predict_image(sample_image, model)
if prediction:
    print("\nПример предсказания:")
    print(f"Изображение: {os.path.basename(sample_image)}")
    print(f"Класс: {prediction['class']}")
    print(f"Вероятность: {prediction['probability']:.4f}")
    print(f"Порог: {prediction['threshold']:.4f}")
    print(f"Уверенность: {prediction['confidence']:.2%}")

# Сохранение модели
model.save('cats_dogs_classifier.h5')
print("\nМодель сохранена как cats_dogs_classifier.h5")


Распределение классов в train: [12500 12500]
Распределение классов в val: [12500]


MemoryError: Unable to allocate 11.2 GiB for an array with shape (20000, 224, 224, 3) and data type float32

In [ ]:
import pandas as pd

# Делаем предсказания на тестовом наборе данных
test_pred = model.predict(X_test)

# Преобразуем предсказания в бинарные метки (0 - кошка, 1 - собака)
# test_pred_labels = (test_pred > 0.5).astype(int).flatten()

# Создаем DataFrame для submission
submission_df = pd.DataFrame({
    # 'ImageId': range(1, len(test_pred_labels) + 1),
    # 'Label': test_pred_labels
    'id': range(1, len(test_pred) + 1),
    'label': test_pred
})

# Сохраняем CSV
submission_path = 'submission.csv'
submission_df.to_csv(submission_path, index=False)
print(f"Файл сохранен: {submission_path}")

NameError: name 'model' is not defined

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.applications.efficientnet import preprocess_input
import os
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import log_loss, classification_report, confusion_matrix, roc_curve, auc
from tensorflow.keras.applications import EfficientNetB0
import seaborn as sns
from datetime import datetime

# Кастомный callback для записи learning rate
class LRTracker(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = tf.keras.backend.get_value(self.model.optimizer.learning_rate)

# Пути к данным
train_dir = "data/train/train"
val_dir = "data/test/test"

# Загрузка и проверка данных
def load_and_validate_data():
    train_images = [os.path.join(train_dir, img) for img in os.listdir(train_dir) if img.endswith(('.jpg', '.png'))]
    val_images = [os.path.join(val_dir, img) for img in os.listdir(val_dir) if img.endswith(('.jpg', '.png'))]

    assert len(train_images) > 0, "Нет изображений в train"
    assert len(val_images) > 0, "Нет изображений в test"

    return train_images, val_images

# Создание генератора данных
def create_generator(image_paths, labels, batch_size=16, target_size=(224, 224), shuffle=True):
    datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rotation_range=25,
        width_shift_range=0.15,
        height_shift_range=0.15,
        shear_range=0.15,
        zoom_range=0.15,
        horizontal_flip=True,
        brightness_range=[0.85, 1.15],
        fill_mode='nearest'
    )
    
    # Конвертируем числовые метки в строковые
    str_labels = ['dog' if label == 1 else 'cat' for label in labels]
    
    # Создаем генератор из массива
    generator = datagen.flow_from_dataframe(
        dataframe=pd.DataFrame({'filename': image_paths, 'class': str_labels}),
        x_col='filename',
        y_col='class',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary',
        shuffle=shuffle
    )
    return generator

# Создание модели
def create_model():
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    for layer in base_model.layers[:3]:
        layer.trainable = False

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(1024, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        BatchNormalization(),
        Dropout(0.5),
        Dense(512, activation='swish', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])

    optimizer = AdamW(learning_rate=0.001, weight_decay=1e-4)
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy',
                 tf.keras.metrics.Precision(name='precision'),
                 tf.keras.metrics.Recall(name='recall'),
                 tf.keras.metrics.AUC(name='auc')]
    )
    return model

# Основной код
train_images, val_images = load_and_validate_data()
train_labels = [1 if "dog" in os.path.basename(img).lower() else 0 for img in train_images]
val_labels = [1 if "dog" in os.path.basename(img).lower() else 0 for img in val_images]

# Создаем генераторы
train_generator = create_generator(train_images, train_labels, batch_size=16)
val_generator = create_generator(val_images, val_labels, batch_size=16, shuffle=False)

# Проверка классов
print("Индексы классов:", train_generator.class_indices)
print("Распределение классов в train:", np.bincount(train_generator.classes))
print("Распределение классов в val:", np.bincount(val_generator.classes))

# Веса классов для балансировки
class_counts = np.bincount(train_generator.classes)
total_samples = len(train_generator.classes)
class_weights = {
    0: total_samples / (2 * class_counts[0]),  # Вес для класса 'cat'
    1: total_samples / (2 * class_counts[1])   # Вес для класса 'dog'
}

model = create_model()

# Callbacks
log_dir = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")
callbacks = [
    EarlyStopping(monitor='val_auc', patience=5, mode='max', verbose=1, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', monitor='val_auc', save_best_only=True, mode='max'),
    ReduceLROnPlateau(monitor='val_auc', factor=0.5, patience=3, min_lr=1e-6, mode='max', verbose=1),
    LRTracker(),
    TensorBoard(log_dir=log_dir)
]

# Обучение модели
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=30,
    validation_data=val_generator,
    validation_steps=len(val_generator),
    callbacks=callbacks,
    class_weight=class_weights,
    verbose=1
)

# Визуализация метрик
def plot_metrics(history):
    plt.figure(figsize=(18, 12))

    metrics = ['loss', 'accuracy', 'precision', 'recall', 'auc']
    for i, metric in enumerate(metrics):
        plt.subplot(2, 3, i+1)
        plt.plot(history.history[metric], label=f'Train {metric}')
        plt.plot(history.history[f'val_{metric}'], label=f'Validation {metric}')
        plt.xlabel('Epochs')
        plt.ylabel(metric)
        plt.legend()

    plt.subplot(2, 3, 6)
    plt.plot(history.history['lr'], label='Learning Rate')
    plt.xlabel('Epochs')
    plt.ylabel('LR')
    plt.yscale('log')
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_metrics(history)

# Оценка модели
def evaluate_model(model, generator):
    # Получаем истинные метки
    y_true = generator.classes
    
    # Делаем предсказания
    generator.reset()
    y_pred = model.predict(generator, steps=len(generator))
    
    # Находим оптимальный порог
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    y_pred_classes = (y_pred > optimal_threshold).astype(int)

    print(f"\nОптимальный порог: {optimal_threshold:.4f}")
    
    # Вычисляем метрики
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred_classes, target_names=['Cat', 'Dog']))
    
    print(f"\nLogLoss: {log_loss(y_true, y_pred):.4f}")
    
    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred_classes)
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Cat', 'Dog'],
                yticklabels=['Cat', 'Dog'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()
    
    # ROC Curve
    roc_auc = auc(fpr, tpr)
    plt.figure()
    plt.plot(fpr, tpr, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()

print("\nОценка на валидационных данных:")
evaluate_model(model, val_generator)

# Сохранение модели
model.save('cats_dogs_classifier.h5')
print("\nМодель сохранена как cats_dogs_classifier.h5")

# Функция для предсказания на новых изображениях
def predict_image(img_path, model, threshold=None):
    try:
        img = cv2.imread(img_path)
        if img is None:
            raise ValueError(f"Не удалось загрузить изображение: {img_path}")
            
        img = cv2.resize(img, (224, 224))
        img = preprocess_input(img)
        img_array = np.expand_dims(img, axis=0)
        
        probability = model.predict(img_array, verbose=0)[0][0]
        
        if threshold is None:
            threshold = 0.5  # Значение по умолчанию
            
        class_name = 'Dog' if probability > threshold else 'Cat'
        confidence = probability if class_name == 'Dog' else 1 - probability
        
        return {
            'class': class_name,
            'probability': float(probability),
            'confidence': float(confidence),
            'threshold': float(threshold)
        }
    except Exception as e:
        print(f"Ошибка при предсказании: {str(e)}")
        return None

# Пример предсказания
if val_images:
    sample_image = val_images[0]
    prediction = predict_image(sample_image, model)
    if prediction:
        print("\nПример предсказания:")
        print(f"Изображение: {os.path.basename(sample_image)}")
        print(f"Класс: {prediction['class']}")
        print(f"Вероятность: {prediction['probability']:.4f}")
        print(f"Уверенность: {prediction['confidence']:.2%}")

Found 25000 validated image filenames belonging to 2 classes.


ValueError: If class_mode="binary" there must be 2 classes. Found 1 classes.